# Gamma Correction of Projected Images

In [2]:
import numpy as np
import cv2
import os

from sfdi.calibration.gamma_calibration import GammaCalibration
from matplotlib import pyplot as plt

## Generate 'intensity_count' number of greyscale images

Interpolates between complete black (0) and complete white (0)

In [3]:
intensity_count = 32

intensities = np.linspace(0, 255, intensity_count)

## <font color='yellow'>*Optional*</font> - Write greyscale images to folder for external capture

In [19]:
width, height = 1280, 720

imgs = [(np.ones((height, width, 3)) * i).astype(np.uint8) for i in intensities] # 3 channels for rgb

#img_out_dir = 'C:\\OneDrive\\OneDrive - The University of Nottingham\\university\\phd\\year1\\Code\\BlenderFPP\\Images\\gamma_calibration\\'
img_out_dir = 'C:\\Users\psydw2\\OneDrive - The University of Nottingham\\university\\phd\\year1\\Code\\BlenderFPP\\Images\\gamma_calibration'

for i, img in enumerate(imgs):
    cv2.imwrite(os.path.join(img_out_dir, f'calibration{i}.jpg'), img)

## Load images from directory

In [6]:
#img_in_dir = 'C:\\OneDrive\\OneDrive - The University of Nottingham\\university\\phd\\year1\\Code\\BlenderFPP\\Images\\gamma_calibration\\'
img_in_dir = 'C:\\OneDrive\\OneDrive - The University of Nottingham\\university\\phd\\year1\\Code\\BlenderFPP\\Images\\gamma_calibration\\'

captured_imgs = []
for i in range(len(intensities)):
    temp = cv2.imread(os.path.join(img_in_dir, f'result{i}.jpg')).astype(np.uint8)
    captured_imgs.append(temp)

print(f'{len(captured_imgs)} images loaded')

32 images loaded


## Calculate the Gamma Curve

Calcuates average pixel value for captured imgs in a region of interest and determines the smallest recognisable 
change in intensity, i.e: a difference larger than delta.

Order determines the polynomial order for the gamma calibration

In [10]:
coeffs, values = GammaCalibration.calculate_curve(captured_imgs, intensities, delta=0.25, order=5)

print(coeffs, values)

[ 2.77781106e-08 -2.63562730e-05  9.88789741e-03 -1.83640281e+00
  1.70092328e+02 -6.21606940e+03] [  8.22580645  16.4516129   24.67741935  32.90322581  41.12903226
  49.35483871  57.58064516  65.80645161  74.03225806  82.25806452
  90.48387097  98.70967742 106.93548387 115.16129032 123.38709677
 131.61290323 139.83870968 148.06451613 156.29032258 164.51612903
 172.74193548 180.96774194 189.19354839 197.41935484 205.64516129]


## Test the calculated calibration curve

In [16]:
def sinusoidal(width, height, freq, phase=0, orientation=(np.pi / 2.0)):
    x, y = np.meshgrid(np.arange(width, dtype=np.uint8), np.arange(height, dtype=np.uint8))

    gradient = np.sin(orientation) * x - np.cos(orientation) * y

    return np.sin(((2.0 * np.pi * gradient) / freq) + phase)

def normalise_image(img):
    return ((img - img.min()) / (img.max() - img.min()))

def generate_images(width: int, height: int, freq: float, orientation=(np.pi / 2.0), n: int=3):
    imgs = []

    for i in range(n):
        img = sinusoidal(width, height, freq, 2.0 * i * np.pi / n, orientation)

        # Normalise and convert to rgb8 range (change to correct datatype)
        img = (normalise_image(img) * 255.0).astype(np.uint8)

        imgs.append(img)

    return imgs

fringes = generate_images(1280, 720, 32.0)

x1, x2 = values.min(), values.max()

corrected_fringes = [gamma.apply_correction(fr.copy(), coeffs, x1, x2) for fr in fringes]


## <font color='yellow'>*Optional*</font> - Save test calibration curve images

In [6]:
path = 'C:\\Users\\psydw2\\OneDrive - The University of Nottingham\\university\\phd\\year1\\Code\\BlenderFPP\\fpp\\'

for i, img in enumerate(fringes):
    cv2.imwrite(os.path.join(path, f'original{i}.jpg'), img)

for i, img in enumerate(corrected_fringes):
    cv2.imwrite(os.path.join(path, f'corrected{i}.jpg'), img)